# Comparision Dashboard

### References : https://github.com/oegedijk/explainerdashboard

#### import libraries

In [43]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm as ltb
import xgboost as xgb
import catboost as catb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from explainerdashboard import ClassifierExplainer, ExplainerDashboard, ExplainerHub
from explainerdashboard.custom import *
import re

#### import the dataset

In [44]:
df = pd.read_csv("./cleaned_data/mental_health_data_2016-2021_ready_for_dashboard.csv")

# drop unnamed columns
df = df.drop('Unnamed: 0', 1)

C:\Users\Imran\AppData\Local\Temp/ipykernel_14408/235644276.py:4: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



#### correct column name using regular expression

In [45]:
arp_data1 = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

#### Create train and test datasets

In [46]:

y=arp_data1["have_mental_illness"]
X=arp_data1.drop("have_mental_illness", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## Preparing models

In [47]:
################################# 1st Model -- CATBOOST ####################################

model_catb_tuned = catb.CatBoostClassifier(depth=10, iterations=30, learning_rate=0.1)
model_catb_tuned.fit(X_train, y_train)
explainer_catb = ClassifierExplainer(
                model_catb_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)


Detected RandomForestClassifier model: Changing class type to RandomForestClassifierExplainer...
Note: model_output=='probability', so assuming that raw shap output of RandomForestClassifier is in probability space...
Generating self.shap_explainer = shap.TreeExplainer(model)


In [48]:
################################# 2nd Model -- XGBOOST ####################################

model_xgb_tuned = xgb.XGBClassifier(depth=4, iterations=10, learning_rate=0.02)

model_xgb_tuned.fit(X_train, y_train)

explainer_xgb = ClassifierExplainer(
                model_xgb_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)


[02:49:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "depth", "iterations" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
Note: model_output=='probability'. For XGBClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a ba

In [49]:
################################# 3rd Model -- LIGHTGBM ####################################

model_lgbm_tuned = ltb.LGBMClassifier(depth=4, iterations=10, learning_rate=0.01)

model_lgbm_tuned.fit(X_train, y_train)

explainer_lgbm = ClassifierExplainer(
                model_lgbm_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)

Note: model_output=='probability'. For LGBMClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...


## Adding comparison components for CatBoost, XGBoost, LightGBM

In [50]:

class ConfusionComparison(ExplainerComponent):
    
    def __init__(self, explainer_catb, explainer_xgb, explainer_lgbm):
        super().__init__(explainer_catb, explainer_lgbm)
        
      
        # 1.Confusion Matrix Component
        self.confmat_cmc_catb = ConfusionMatrixComponent(explainer_catb, subtitle='CatBoost', percentage=True,
                            count=False, binary=False,
                            hide_selector=True, hide_percentage=True)
        self.confmat_cmc_xgb = ConfusionMatrixComponent(explainer_xgb, subtitle='XGBoost', 
                            hide_selector=True, hide_percentage=True)
        self.confmat_cmc_lgbm = ConfusionMatrixComponent(explainer_lgbm, subtitle='LightGBM',
                            hide_selector=True, hide_percentage=True)
        
        # 2.Precision Component
        self.confmat_pc_catb = PrecisionComponent(explainer_catb, subtitle='CatBoost',
                            hide_selector=True, hide_percentage=True)
        self.confmat_pc_xgb = PrecisionComponent(explainer_xgb, subtitle='XGBoost',
                            hide_selector=True, hide_percentage=True)
        self.confmat_pc_lgbm = PrecisionComponent(explainer_lgbm, subtitle='LightGBM',
                            hide_selector=True, hide_percentage=True)     
    
        #Classification Plot Component
        self.confmat_cc_catb = ClassificationComponent(explainer_catb, title='Classification Plot', name=None, subtitle='CatBoost', 
                                                 pos_label=None, percentage=True)
        
        self.confmat_cc_xgb = ClassificationComponent(explainer_xgb, title='Classification Plot', name=None, subtitle='XGBoost', 
                                                 pos_label=None, percentage=True )
        
        self.confmat_cc_lgbm = ClassificationComponent(explainer_lgbm, title='Classification Plot', name=None, subtitle='LightGBM', 
                                                 pos_label=None,  percentage=True)
        
        #LiftCurveComponent
        self.confmat_lcc_catb = LiftCurveComponent(explainer_catb, title='Lift Curve', subtitle='CatBoost', hide_title=False,
                                            pos_label=None, wizard=True)
        
        self.confmat_lcc_xgb = LiftCurveComponent(explainer_xgb, title='Lift Curve', subtitle='XGBoost', hide_title=False,
                                            pos_label=None, wizard=True)
        
        self.confmat_lcc_lgbm =  LiftCurveComponent(explainer_lgbm, title='Lift Curve', subtitle='LightGBM', hide_title=False,
                                            wizard=True)
        
        #RocAucComponent
        self.confmat_rac_catb = RocAucComponent(explainer_catb, title='ROC AUC Plot', name=None, subtitle='CatBoost',
                        pos_label=None)
        
        self.confmat_rac_xgb =RocAucComponent(explainer_xgb, title='ROC AUC Plot', name=None, subtitle='XGBoost',
                        pos_label=None)
        
        self.confmat_rac_lgbm =  RocAucComponent(explainer_lgbm, title='ROC AUC Plot', name=None, subtitle='LightGBM', 
                        pos_label=None)
        
        #PrAucComponent
        self.confmat_pac_catb = PrAucComponent(explainer_catb, title='PR AUC Plot', name=None, subtitle='CatBoost',
                        pos_label=None)
        
        self.confmat_pac_xgb =PrAucComponent(explainer_xgb, title='PR AUC Plot', name=None, subtitle='XGBoost',
                        pos_label=None)
        
        self.confmat_pac_lgbm =  PrAucComponent(explainer_lgbm, title='PR AUC Plot', name=None, subtitle='LightGBM', 
                        pos_label=None)
        
        
       
        #PredictionSummaryComponent
        self.confmat_cpsc_catb = ClassifierPredictionSummaryComponent(explainer_catb, subtitle='CatBoost',hide_star_explanation= False, 
                        title='Prediction Summary of CatBoost', 
                        name=None, index_dropdown=True, round=3,
                        hide_selector=False, pos_label=None, index=None)
        
        self.confmat_cpsc_xgb = ClassifierPredictionSummaryComponent(explainer_xgb, subtitle='XGBoost', hide_star_explanation= False,
                        title='Prediction Summary of XGBoost', 
                        name=None, index_dropdown=True, round=3,
                        hide_selector=False, pos_label=None, index=None)
        
        self.confmat_cpsc_lgbm = ClassifierPredictionSummaryComponent(explainer_lgbm, subtitle='LightGBM', hide_star_explanation= False,
                        title='Prediction Summary of LightGBM', 
                        name=None, index_dropdown=True, round=3,
                        hide_selector=False, pos_label=None, index=None)
        
        
    # Making layout for the comparision tab to display the models parallel. 
    def layout(self):
        return dbc.Container([
            
            
            #Confusion matrix component
            dbc.Row([
                dbc.Col([
                    self.confmat_cmc_catb.layout()
                ]),
                dbc.Col([
                    self.confmat_cmc_xgb.layout()
                ]),
                dbc.Col([
                    self.confmat_cmc_lgbm.layout()
                ])
            ]),
            
            #Precision plot component
            dbc.Row([
                dbc.Col([
                    self.confmat_pc_catb.layout()
                ]),
                dbc.Col([
                    self.confmat_pc_xgb.layout()
                ]),
                dbc.Col([
                    self.confmat_pc_lgbm.layout()
                ])
            ]),
            
            
            #Classification Plot Component
            dbc.Row([
                dbc.Col([
                    self.confmat_cc_catb.layout()
                ]),
                dbc.Col([
                    self.confmat_cc_xgb.layout()
                ]),
                dbc.Col([
                    self.confmat_cc_lgbm.layout()
                ])
            ]),
            
            #LiftCurveComponent
            dbc.Row([
                dbc.Col([
                    self.confmat_lcc_catb.layout()
                ]),
                dbc.Col([
                    self.confmat_lcc_xgb.layout()
                ]),
                dbc.Col([
                    self.confmat_lcc_lgbm.layout()
                ])
            ]),
            
            #RocAucComponent
            dbc.Row([
                dbc.Col([
                    self.confmat_rac_catb.layout()
                ]),
                dbc.Col([
                    self.confmat_rac_xgb.layout()
                ]),
                dbc.Col([
                    self.confmat_rac_lgbm.layout()
                ])
            ]),
            
            #PR Auc Component
            dbc.Row([
                dbc.Col([
                    self.confmat_pac_catb.layout()
                ]),
                dbc.Col([
                    self.confmat_pac_xgb.layout()
                ]),
                dbc.Col([
                    self.confmat_pac_lgbm.layout()
                ])
            ]),
            
           
            
            #PredictionSummaryComponent
            dbc.Row([
                dbc.Col([
                    self.confmat_cpsc_catb.layout()
                ]),
                dbc.Col([
                    self.confmat_cpsc_xgb.layout()
                ]),
                dbc.Col([
                    self.confmat_cpsc_lgbm.layout()
                ])
            ])   
            
        ])

        
        


#### Creating the comparison dashboard

In [51]:
comparision_dashboard = ConfusionComparison(explainer_catb, explainer_xgb, explainer_lgbm)

## Launching the Dashboard on localhost

In [ ]:
ExplainerDashboard(explainer_catb,
                   comparision_dashboard, title = "\n \n Comparison Dashboard for CatBoost, XGBoost & LightGBM models on Mental Health. \n", ).run(port=8051)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://192.168.178.22:8051
Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8051/

Dash is running on http://0.0.0.0:8

 * Running on all addresses.
 * Running on http://192.168.178.22:8051/ (Press CTRL+C to quit)
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET / HTTP/1.1" 200 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET /assets/bootstrap.min.css?m=1651599647.3112018 HTTP/1.1" 304 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET /_dash-layout HTTP/1.1" 200 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
192.168.178.22 - - [18/Jun/2022 02:55:35] "POST /_dash-update-component HTTP/1.1" 204 -
192.168.178.22 - - [18/Jun/2022 